# PoC: WeCookio multiagent CrewAI INC

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Process.Hierarquical with Coordinator

### Installing Required Packages with uv

In [1]:
# Install required packages using uv
!uv add ipykernel crewai crewai-tools boto3 botocore pyyaml ipywidgets

Resolved 224 packages in 0.74ms
Audited 213 packages in 0.07ms


### Import Libraries and Load Configuration

In [2]:
import yaml
from crewai import Crew, Agent, Task, Process
#import boto3
import json

print("Libraries imported successfully!")

Libraries imported successfully!


HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x74c46539da90>, 'Connection to telemetry.crewai.com timed out. (connect timeout=29.9999942779541)'))
HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x74c457e3b750>, 'Connection to telemetry.crewai.com timed out. (connect timeout=29.99998950958252)'))


### Loading AWS Enviroment variables

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [4]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "crew/latest/"
# Load the integration configuration
with open(prefix+'config/agent-coordinate/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/agent-coordinate/crew_config.yaml', 'r') as file:
    crew_config = yaml.safe_load(file)
with open(prefix+'config/agent-coordinate/agents.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/agent-coordinate/tasks.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(crew_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

/home/e2its/dev/weCookio/researches
Configuration files loaded successfully!
Integration config keys: ['aws_bedrock', 'database', 'api_endpoints']
Crew Orchestration keys: ['crew', 'agents', 'tasks', 'integration_config', 'model_assignment', 'execution', 'memory', 'input_schema', 'output_schema', 'cost_optimization', 'performance_metrics', 'quality_standards', 'monitoring', 'example_input', 'example_output']
Agents config keys: ['Culinary Coordinator', 'Dietary Compliance Specialist', 'Ingredient Substitution Expert', 'Culinary Experience Optimizer', 'Quality Assurance Chef']
Tasks config keys: ['Enhance Recipe']


###  Model Selection based on complexity

In [5]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

High complexity model: bedrock/amazon.nova-premier-v1:0
Medium complexity model: bedrock/amazon.nova-pro-v1:0
Low complexity model: bedrock/amazon.nova-lite-v1:0
Ultra-Low complexity model: bedrock/amazon.nova-micro-v1:0


### Create Agents with Model-Specific Configuration

In [6]:
from crewai import LLM
from pprint import pprint

def create_agent_with_model(agent_config, model_config):
    """Create an agent with specific model configuration"""
    llm = LLM(
        model=model_config["model_id"],
        region_name=AWS_DEFAULT_REGION,
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000),
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        api_key=AWS_SECRET_ACCESS_KEY
    )
    return Agent(
        role=agent_config["role"],
        goal=agent_config["goal"],
        backstory=agent_config["backstory"],
        verbose=agent_config.get("verbose", True),
        allow_delegation=agent_config.get("allow_delegation", False),
        prompt_template=agent_config.get("prompt", None),
        reasoning=agent_config.get("reasoning", False),
        max_reasoning_attempts=agent_config.get("max_reasoning_attempts", 3),
        #response_template="{{ .Response }}",
        max_iterations=model_config.get("model_kwargs").get("max_iterations", 3),
        max_retry_limit=model_config.get("model_kwargs").get("max_retry_limit", 2),
        max_execution_time=model_config.get("model_kwargs").get("max_execution_time", 30),
        llm=llm
    )

def print_agent_info(agent, model_config) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.id}\n"
        f"Agent Role: {agent.role}\n"
        f"Using model: {model_config.get('model_id')}"
    )
    print("\n")

In [7]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent, model_config)


#agent_name = "Dietary Compliance Specialist"
agent_config = agents_config["Dietary Compliance Specialist"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
dietary_compliance_specialist_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(dietary_compliance_specialist_agent, model_config)

#agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent, model_config)

#agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent, model_config)

#agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent, model_config)


Created agent: a1e5fbd0-4530-4a20-969f-be4bd03b63d4
Agent Role: Master Chef & Project Manager
Using model: bedrock/amazon.nova-lite-v1:0


Created agent: 6cd48b5c-433e-4796-b616-3f373e66600d
Agent Role: Nutritionist & Food Safety Expert
Using model: bedrock/amazon.nova-micro-v1:0


Created agent: 673ad016-4577-4a89-bd09-5fcd922facc5
Agent Role: Culinary Innovation Specialist
Using model: bedrock/amazon.nova-pro-v1:0


Created agent: ad52d943-c4da-427f-b5e3-4549ea7ad01e
Agent Role: Gastronomic Enhancement Specialist
Using model: bedrock/amazon.nova-lite-v1:0


Created agent: 831d0e00-84ad-4343-8574-1380226355dc
Agent Role: Final Validation & Presentation Expert
Using model: bedrock/amazon.nova-lite-v1:0




### Create Tasks with Complexity-Based Model Selection

In [8]:
def create_task_with_agent(task_config, agent):
    """Create a task with the appropriate agent based on complexity"""
    return Task(
        description=task_config["description"],
        expected_output=task_config["expected_output"],
        agent=agent,
        async_execution=task_config.get("async_execution", False)
    )

def print_task_info(task) -> None:
    """Print information about a created task."""
    print(f"Created task: {task.id}")
    print(f"Task description: {task.description}")
    print(f"Assigned to agent: {task.agent.role}")
    print("\n")

In [9]:
# task_name = "Enhance Recipe"
task_config = tasks_config["Enhance Recipe"]
enhance_recipe_task = create_task_with_agent(task_config, eval(task_config["agent"]))
print_task_info(enhance_recipe_task)


Created task: 22e6c94f-339e-4bcb-b545-f04d9647209f
Task description: You are a culinary expert tasked with enhancing and adapting recipes for dietary compliance while maintaining authentic flavors and textures.

## TASK OBJECTIVE
Transform the {{food_name}} recipe to be 100% compliant with dietary restrictions while maximizing culinary experience and preserving authentic flavor/texture characteristics.

## CRITICAL REQUIREMENTS
1. **DIETARY COMPLIANCE**: Zero tolerance for dietary restriction violations - this is your highest priority
2. **CULINARY EXCELLENCE**: Maximize taste, texture, and overall dining experience
3. **AUTHENTICITY**: Maintain the original recipe's cultural and flavor profile
4. **LANGUAGE**: Provide ALL output exclusively in {{output_language}} - no mixed languages permitted

## RECIPE CONTEXT
- **Recipe Name**: {{food_name}}
- **Servings**: {{servings}}
- **Dietary Restrictions**: {{intolerances}}, {{exclusions}}
- **Preferences**: {{preferences}}
- **Target Countr

### Create Crew 

In [10]:
# Create a simple crew for demonstration
embedder={
        "provider": "bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v2:0"
        }
    }

crew = Crew(
    #agents=[culinary_coordinator_agent,dietary_compliance_specialist_agent, culinary_experience_optimizer_agent, ingredient_substitution_expert_agent, quality_assurance_chef_agent],
    #tasks=[inital_recipe_analysis_task, dietary_compliance_validation_task, culinary_experience_enhancement_task, final_quality_assurance_task, ingredient_substitution_strategy_task],
    agents=[culinary_coordinator_agent,culinary_experience_optimizer_agent, ingredient_substitution_expert_agent, quality_assurance_chef_agent],
    tasks=[enhance_recipe_task],
    verbose=True,
    memory=False, 
    #memory=True,
    #embedder=embedder,
    #process=eval(crew_config.get("execution").get("process")),
    #manager_agent=culinary_coordinator_agent,
    cache=crew_config.get("cache", True)
)

print("Crew created successfully!")
print("Flow model: ", crew_config.get("execution").get("process"))
print(f"Number of agents: {len(crew.agents)}")
print(f"Number of tasks: {len(crew.tasks)}")
print(f"Memory enabled: {crew.memory}")
print(f"Cache enabled: {crew.cache}")

Crew created successfully!
Flow model:  Process.hierarchical
Number of agents: 4
Number of tasks: 1
Memory enabled: False
Cache enabled: True


### Recipe Parameters

In [11]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Milk",
        "Eggs",
        "Beef",
        "Corn",
        "Gluten",
        "Rice",
        "Garlic"
    ],
    "exclusions": ["Pepper"],
    "preferences": [],
    "output_language": "es",
    "country": "ES"
}

print("Sample request for analysis:")
print(json.dumps(recipe_request, indent=2))

Sample request for analysis:
{
  "food_name": "Canelones de ternera XXL al Pedro Ximenez",
  "servings": 4,
  "intolerances": [
    "Milk",
    "Eggs",
    "Beef",
    "Corn",
    "Gluten",
    "Rice",
    "Garlic"
  ],
  "exclusions": [
    "Pepper"
  ],
  "preferences": [],
  "output_language": "es",
  "country": "ES"
}


### Run the Crew (Requires AWS Credentials)

In [12]:
# Uncomment the following lines if you have AWS credentials configured
# and want to actually run the crew

try:
    result = crew.kickoff(inputs=recipe_request)
    print("\nCrew execution result:")
    print(result)
except Exception as e:
    print(f"Error running crew: {e}")
    print("Make sure you have AWS credentials configured and the required permissions.")

#print("Notebook execution completed successfully!")
#print("\nTo run the actual crew:")
#print("1. Configure AWS credentials")
#print("2. Ensure Bedrock access permissions")
#print("3. Uncomment the execution code above")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 769d08eb-052b-45e8-a168-74e5d5ef8aed                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Master Chef & Project Manager                                                                           │
│                                                                                                                 │
│  Task: You are a culinary expert tasked with enhancing and adapting recipes for dietary compliance while        │
│  maintaining authentic flavors and textures.                                                                    │
│                                                                                                                 │
│  ## TASK OBJECTIVE                                                                                              │
│  Transform the {Canelones de ternera XXL al Pedro Ximenez} recipe to be 100% compliant with dietary             │
│  restrictions while maximizing culinary experience and preserving authentic flavor/texture characteristics.     │
│                                                                                                                 │
│  ## CRITICAL REQUIREMENTS                                                                                       │
│  1. **DIETARY COMPLIANCE**: Zero tolerance for dietary restriction violations - this is your highest priority   │
│  2. **CULINARY EXCELLENCE**: Maximize taste, texture, and overall dining experience                             │
│  3. **AUTHENTICITY**: Maintain the original recipe's cultural and flavor profile                                │
│  4. **LANGUAGE**: Provide ALL output exclusively in {es} - no mixed languages permitted                         │
│                                                                                                                 │
│  ## RECIPE CONTEXT                                                                                              │
│  - **Recipe Name**: {Canelones de ternera XXL al Pedro Ximenez}                                                 │
│  - **Servings**: {4}                                                                                            │
│  - **Dietary Restrictions**: {['Milk', 'Eggs', 'Beef', 'Corn', 'Gluten', 'Rice', 'Garlic']}, {['Pepper']}       │
│  - **Preferences**: {[]}                                                                                        │
│  - **Target Country**: {ES}                                                                                     │
│  - **Output Language**: {es}                                                                                    │
│                                                                                                                 │
│  ## MEASUREMENT ADAPTATION RULES                                                                                │
│  - **Metric System** (most countries): grams, milliliters, Celsius                                              │
│  - **Imperial System** (US): ounces, cups, Fahrenheit                                                           │
│  - Adapt all measurements to {ES} standards                                                                     │
│  - Ensure temperature and weight measurements are country-appropriate                                           │
│                                                                                                                 │
│  ## REQUIRED OUTPUT FORMAT                                                                                      │
│  Provide the enhanced recipe in the following structured format (ALL in {es}):                                  │
│                                                                                                                 │
│  ### Recipe Title                                      

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Culinary Innovation Specialist                                                                          │
│                                                                                                                 │
│  Task: ¿Cuáles son las mejores alternativas para los ingredientes restringidos en la receta de Canelones de     │
│  ternera XXL al Pedro Ximenez para cumplir con las restricciones dietéticas de leche, huevos, ternera, maíz,    │
│  gluten, arroz, ajo y pimienta?                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Master Chef & Project Manager                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│                                                                                                                 │
│  **Canelones de Jackfruit XXL al Pedro Ximenez**                                                                │
│                                                                                                                 │
│  **Ingredientes:**                                                                                              │
│                                                                                                                 │
│  *Para el relleno:*                                                                                             │
│  - 1 lata de jackfruit texturizado (aprox. 400g), escurrido y desmenuzado                                       │
│  - 1 cebolla grande, picada finamente                                                                           │
│  - 2 zanahorias, peladas y picadas finamente                                                                    │
│  - 2 dientes de ajo, en polvo de cebolla o asafétida                                                            │
│  - 1 taza de caldo de verduras (puede ser de verduras o de champiñones)                                         │
│  - 1 taza de vino Pedro Ximenez                                                                                 │
│  - 1 taza de leche de almendras o de avena                                                                      │
│  - 2 cucharadas de aceite de oliva                                                                              │
│  - 1 cucharadita de pimienta de cayena o pimienta blanca                                                        │
│  - Sal y pimienta al gusto                                                                                      │
│                                                                                                                 │
│  *Para la salsa de tomate:*                                                                                     │
│  - 2 latas de tomate triturado                                                                                  │
│  - 1 cebolla grande, picada finamente                                                                           │
│  - 2 dientes de ajo, en polvo de cebolla o asafétida                                                            │
│  - 2 cucharadas de aceite de oliva                                                                              │
│  - 1 cucharadita de azúcar                                                                                      │
│  - Sal y pimienta al gusto                                                                                      │
│                                                                                                                 │
│  *Para el béchamel:*                                                                                            │
│  - 2 cucharadas de aceite de oliva                                                                              │
│  - 2 cucharadas de mezcla de harinas sin gluten                                                                 │
│  - 1 taza de leche de almendras o de avena             

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 22e6c94f-339e-4bcb-b545-f04d9647209f                                                                     │
│  Agent: Master Chef & Project Manager                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 769d08eb-052b-45e8-a168-74e5d5ef8aed                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│                                                                                                                 │
│  **Canelones de Jackfruit XXL al Pedro Ximenez**                                                                │
│                                                                                                                 │
│  **Ingredientes:**                                                                                              │
│                                                                                                                 │
│  *Para el relleno:*                                                                                             │
│  - 1 lata de jackfruit texturizado (aprox. 400g), escurrido y desmenuzado                                       │
│  - 1 cebolla grande, picada finamente                                                                           │
│  - 2 zanahorias, peladas y picadas finamente                                                                    │
│  - 2 dientes de ajo, en polvo de cebolla o asafétida                                                            │
│  - 1 taza de caldo de verduras (puede ser de verduras o de champiñones)                                         │
│  - 1 taza de vino Pedro Ximenez                                                                                 │
│  - 1 taza de leche de almendras o de avena                                                                      │
│  - 2 cucharadas de aceite de oliva                                                                              │
│  - 1 cucharadita de pimienta de cayena o pimienta blanca                                                        │
│  - Sal y pimienta al gusto                                                                                      │
│                                                                                                                 │
│  *Para la salsa de tomate:*                                                                                     │
│  - 2 latas de tomate triturado                                                                                  │
│  - 1 cebolla grande, picada finamente                                                                           │
│  - 2 dientes de ajo, en polvo de cebolla o asafétida                                                            │
│  - 2 cucharadas de aceite de oliva                                                                              │
│  - 1 cucharadita de azúcar                                                                                      │
│  - Sal y pimienta al gusto                                                                                      │
│                                                                                                                 │
│  *Para el béchamel:*                                                                                            │
│  - 2 cucharadas de aceite de oliva                                                                              │
│  - 2 cucharadas de mezcla de harinas sin gluten       


Crew execution result:
```

**Canelones de Jackfruit XXL al Pedro Ximenez**

**Ingredientes:**

*Para el relleno:*
- 1 lata de jackfruit texturizado (aprox. 400g), escurrido y desmenuzado
- 1 cebolla grande, picada finamente
- 2 zanahorias, peladas y picadas finamente
- 2 dientes de ajo, en polvo de cebolla o asafétida
- 1 taza de caldo de verduras (puede ser de verduras o de champiñones)
- 1 taza de vino Pedro Ximenez
- 1 taza de leche de almendras o de avena
- 2 cucharadas de aceite de oliva
- 1 cucharadita de pimienta de cayena o pimienta blanca
- Sal y pimienta al gusto

*Para la salsa de tomate:*
- 2 latas de tomate triturado
- 1 cebolla grande, picada finamente
- 2 dientes de ajo, en polvo de cebolla o asafétida
- 2 cucharadas de aceite de oliva
- 1 cucharadita de azúcar
- Sal y pimienta al gusto

*Para el béchamel:*
- 2 cucharadas de aceite de oliva
- 2 cucharadas de mezcla de harinas sin gluten
- 1 taza de leche de almendras o de avena
- Una pizca de nuez moscada
- Sal al gust